# Topic 2: Basic Document Classification

## Preliminaries 
Run this cell.

In [2]:
import sys
sys.path.append(r'\\ad.susx.ac.uk\ITS\TeachingResources\Departments\Informatics\LanguageEngineering\resources')
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections
from collections import defaultdict,Counter
from itertools import zip_longest
from IPython.display import display
from random import seed
get_ipython().magic('matplotlib inline')
import random
import math
import matplotlib.pylab as pylab
%matplotlib inline
params = {'legend.fontsize': 'large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'large',
         'axes.titlesize':'large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
pylab.rcParams.update(params)
from pylab import rcParams
from operator import itemgetter, attrgetter, methodcaller
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
import csv

## Overview 
This topic (Topic 2) and Topic 3 concern the task of sentiment analysis. You will be using a corpus of **book reviews** within an **Amazon review corpus**.

You be exploring various techniques that can be used to classify the sentiment of Amazon book reviews as either positive or negative. 

You will be developing your own **Word List** classifiers and then comparing them to the **NLTK Naïve Bayes** classifier.

## Creating training and testing sets
During the next two lab sessions you will be training and testing various document classifiers. It is essential that the data used in the testing phase is not used during the training phase, since this can lead to overestimating performance. 

We now introduce the `split_data` function (defined in the cell below) which can be used to get separate **training** and **testing** sets.

### Exercise
Look through the code in the following cell, reading the comments and making sure that you understand each line.

In [3]:
from random import sample # have a look at https://docs.python.org/3/library/random.html to see what random.sample does
from sussex_nltk.corpus_readers import AmazonReviewCorpusReader

 
def split_data(data, ratio=0.7): # when the second argument is not given, it defaults to 0.7
    """
    Given corpus generator and ratio:
     - partitions the corpus into training data and test data, where the proportion in train is ratio,

    :param data: A corpus generator.
    :param ratio: The proportion of training documents (default 0.7)
    :return: a pair (tuple) of lists where the first element of the 
            pair is a list of the training data and the second is a list of the test data.
    """
    
    data = list(data) # data is a generator, so this puts all the generated items in a list
 
    n = len(data)  #Found out number of samples present
    
    train_indices = sample(range(n), int(n * ratio))          #Randomly select training indices
    test_indices = list(set(range(n)) - set(train_indices))   #Randomly select testing indices
 
    train = [data[i] for i in train_indices]           #Use training indices to select data
    test = [data[i] for i in test_indices]             #Use testing indices to select data
 
    return (train, test)                       #Return split data
 
#Create an Amazon corpus reader pointing at only book reviews
book_reader = AmazonReviewCorpusReader().category("book")

#The following two lines use the documents function on the Amazon corpus reader. 
#This returns a generator over reviews in the corpus. 
#Each review is an instance of a Python class called AmazonReview. 
#An AmazonReview object contains all the data about a review.
pos_train, pos_test = split_data(book_reader.positive().documents())
neg_train, neg_test = split_data(book_reader.negative().documents())

#You can also combine the training data
train = pos_train + neg_train
test = pos_test + neg_test

Sussex NLTK root directory is \\ad.susx.ac.uk\ITS\TeachingResources\Departments\Informatics\LanguageEngineering\resources
1000
1000


### Exercise
Make a copy of the cell above and position it below this cell. Then adapt this code so that it splits the book review corpus in various ways (i.e. different test/training ratios), and by measuring the size of the resulting splits, check that the size of both splits match the specified ratios.

In [4]:
# %load solutions/split_checker
from random import sample # have a look at https://docs.python.org/3/library/random.html to see what random.sample does
from sussex_nltk.corpus_readers import AmazonReviewCorpusReader

 
def split_data(data, ratio=0.7): # when the second argument is not given, it defaults to 0.7
    """
    Given corpus generator and ratio:
     - partitions the corpus into training data and test data, where the proportion in train is ratio,

    :param data: A corpus generator.
    :param ratio: The proportion of training documents (default 0.7)
    :return: a pair (tuple) of lists where the first element of the 
            pair is a list of the training data and the second is a list of the test data.
    """
    
    data = list(data) # data is a generator, so this puts all the generated items in a list
 
    n = len(data)  #Found out number of samples present
    train_indices = sample(range(n), int(n * ratio))          #Randomly select training indices
    test_indices = list(set(range(n)) - set(train_indices))   #Randomly select testing indices
 
    train = [data[i] for i in train_indices]           #Use training indices to select data
    test = [data[i] for i in test_indices]             #Use testing indices to select data
 
    return (train, test)                       #Return split data
 
#Create an Amazon corpus reader pointing at only book reviews
book_reader = AmazonReviewCorpusReader().category("book")
ratios = [0.1,0.5,0.9, 0.34]
for ratio in ratios:
    pos_train, pos_test = split_data(book_reader.positive().documents(),ratio)
    neg_train, neg_test = split_data(book_reader.negative().documents(),ratio)
    print(ratio,len(pos_train),len(pos_test))
    

0.1 100 900
0.5 500 500
0.9 900 100
0.34 340 660


## Creating word lists
The next section will explain how to use a sentiment classifier that bases its decisions on word lists. The classifier requires a list of words indicating positive sentiment, and a second list of words indicating negative sentiment. Given positive and negative word lists, a document's overall sentiment is determined based on counts of occurrences of words that occur in the two lists. In this section we are concerned with the creation of the word lists. We will be considering both hand-crafted lists and automatically generated lists.

### Exercise

- Create a reasonably long hand-crafted list of words that you think indicate positive sentiment.
- Create a reasonably long hand-crafted list of words that indicate negative sentiment.

Use the following cells to store these lists in the variables `my_positive_word_list` and `my_negative_word_list`.

In [5]:
my_positive_word_list = ["good","great","lovely","wonderful","marvellous","perfect","spectacular","best","beautiful","cool"] # put your own list here
my_negative_word_list = ["bad","terrible","awful","disgusting","worst","unpleasant","poor","boring"] # put your own list here

Next, you should try to derive word lists from the data. One way to do this, is to use the most frequent words in positive reviews as your positive list, and the most frequent words in negative reviews as your negative list. This can be done with the [NLTK <code style="background-color: #F5F5F5;">FreqDist</code>](http://www.nltk.org/api/nltk.html#module-nltk.probability) object. 

### Exercise
Make sure that you understand the code in the cell below.

In [6]:
from nltk.probability import FreqDist # see http://www.nltk.org/api/nltk.html#module-nltk.probability
from sussex_nltk.corpus_readers import AmazonReviewCorpusReader
from functools import reduce # see https://docs.python.org/3/library/functools.html

#Helper function. Given a list of reviews, return a list of all the words in those reviews
#To understand this look at the description of functools.reduce in https://docs.python.org/3/library/functools.html
def get_all_words(amazon_reviews):
    #print(reduce(lambda words,review: words + review.words(), amazon_reviews, []))
    return reduce(lambda words,review: words + review.words(), amazon_reviews, [])

#A frequency distribution over all words in positive book reviews
pos_freqdist = FreqDist(get_all_words(pos_train))
neg_freqdist = FreqDist(get_all_words(neg_train))
pos_freqdist

FreqDist({'Ms.': 5,
          'Roddick': 1,
          'has': 214,
          'managed': 4,
          'to': 1496,
          'put': 36,
          'together': 14,
          'some': 109,
          'important': 36,
          'issues': 13,
          ',': 2741,
          'outstanding': 1,
          'commentators': 1,
          'and': 1766,
          'good': 107,
          'information': 38,
          'design': 11,
          'in': 899,
          'one': 206,
          'place': 24,
          'with': 412,
          'this': 559,
          'book': 705,
          '.': 2729,
          'I': 958,
          'read': 226,
          'it': 623,
          'through': 74,
          'from': 223,
          'cover-to-cover': 1,
          '(': 184,
          'seldom': 3,
          'do': 104,
          ')': 193,
          'found': 61,
          'myself': 23,
          'both': 44,
          'enlightened': 1,
          'inspired': 5,
          'recommend': 55,
          'highly': 36,
          'was': 322,
          'g

### Exercise
In the blank code cell below write code that uses the frequency lists, `pos_freqdist` and `neg_freqdist`, created in the above cell and `my_positive_word_list` and `my_negative_word_list` that you manually created earlier to determine whether or not the review data conforms to your expectations. In particular, whether:
- the words you expected to indicate positive sentiment actually occur more frequently in positive reviews than negative reviews
- the words you expected to indicate negative sentiment actually occur more frequently in negative reviews than positive reviews.

Display your findings in a table using pandas.

In [7]:
def check_expectations(wordlist, pos_freq_dist, neg_freq_dist):
    match_freq = [pos_freq_dist[word] for word in wordlist]
    mismatch_freq = [neg_freq_dist[word] for word in wordlist]
    expected = [match_freq[i] > mismatch_freq[i] for i in range(len(wordlist))]
    df = pd.DataFrame(list(zip_longest(wordlist, match_freq, mismatch_freq, expected)), columns = ["word", "match", "mismatch", "expected"])
    display(df)
    
check_expectations(my_positive_word_list, pos_freqdist, neg_freqdist)
check_expectations(my_negative_word_list, pos_freqdist, neg_freqdist)


,word,match,mismatch,expected
0,good,107,112,False
1,great,93,54,True
2,lovely,3,4,False
3,wonderful,26,11,True
4,marvellous,0,0,False
5,perfect,13,6,True
6,spectacular,2,0,True
7,best,75,34,True
8,beautiful,15,6,True
9,cool,2,0,True


,word,match,mismatch,expected
0,bad,9,44,False
1,terrible,5,10,False
2,awful,1,4,False
3,disgusting,1,1,False
4,worst,5,18,False
5,unpleasant,0,2,False
6,poor,11,17,False
7,boring,4,27,False


In [9]:
### %load solutions/check_expectations_partial
def check_expectations(word_list,freqdist1,freqdist2,headers):
    match_freq = [freqdist1[word] for word in word_list]
    mismatch_freq = [freqdist2[word] for word in word_list]
    as_expected = [match_freq[i]>mismatch_freq[i] for i in range(len(word_list))]
    headers.append('Expected?')
    datadict = {headers[0] : word_list,
                headers[1] : match_freq,
                headers[2] : mismatch_freq,
                headers[3] : as_expected}
    df = pd.DataFrame(datadict,columns=headers)
    display(df,"\n")


In [7]:
# %load solutions/check_expectations

### Exercise
The cell below is a copy of the cell that appeared just before the last exercise.

Add two functions to this code.

- `most_frequent_words` - this function should take two arguments: a frequency distribution and a natural number, k. It should return the top k most frequent  words in the frequency distribution. You can use the `most_common` method for this, but you will need to aware of what this method returns.
- `words_above_threshold` - this function also takes two arguments: a frequency distribution and a natural number, k. It should return all of the words that have a frequency greater than k.

Remove punctuation and stopwords from consideration. You can re-use code from near the end of Topic 1 notebook.
Using the training data, create two sets of positive and negative word lists using these functions. 

In [10]:
## from nltk.probability import FreqDist # see http://www.nltk.org/api/nltk.html#module-nltk.probability
from nltk.probability import FreqDist # see http://www.nltk.org/api/nltk.html#module-nltk.probability
from sussex_nltk.corpus_readers import AmazonReviewCorpusReader
from functools import reduce # see https://docs.python.org/3/library/functools.html
from nltk.corpus import stopwords

stopwords = stopwords.words('english')
def remove_stopwords_and_punctuation(words):
    return [w for w in words if w.isalpha() and w not in stopwords]

#Helper function. Given a list of reviews, return a list of all the words in those reviews
#To understand this look at the description of functools.reduce in https://docs.python.org/3/library/functools.html
def get_all_words(amazon_reviews):
    return reduce(lambda words,review: words + review.words(), amazon_reviews, [])
#A frequency distribution over all words in positive book reviews
pos_freqdist = FreqDist(remove_stopwords_and_punctuation(get_all_words(pos_train)))
neg_freqdist = FreqDist(remove_stopwords_and_punctuation(get_all_words(neg_train)))

def most_frequent_words(freq_dist, k):
    return [word for word, count in freq_dist.most_common(k)]               

def words_above_threshold(freq_dist, k):
    return [word for word in freq_dist if freq_dist[word] > k] 

top_pos = most_frequent_words(pos_freqdist,100)
top_neg = most_frequent_words(neg_freqdist,100)
above_pos = words_above_threshold(pos_freqdist,100)
above_neg = words_above_threshold(neg_freqdist,100)
display(pd.DataFrame(list(zip_longest(top_pos,top_neg,above_pos,above_neg)),columns=["TopPos","TopNeg","AbovePos","AboveNeg"]))

,TopPos,TopNeg,AbovePos,AboveNeg
0,I,I,good,I
1,book,book,one,time
2,The,The,book,book
3,read,read,I,could
4,This,one,read,read
5,one,like,The,The
6,It,would,It,This
7,story,It,work,good
8,would,This,many,like
9,life,much,This,story


In [14]:
# %load solutions/make_word_lists


## Creating a word list based classifier
Now you have a number of word lists for use with a classifier. The following code can be used as the basis for creating a word list based classifier.

In [11]:
from nltk.classify.api import ClassifierI
import random

class SimpleClassifier(ClassifierI): 

    def __init__(self, pos, neg): 
        self._pos = pos 
        self._neg = neg 

    def classify(self, words): 
        score = 0
        
        # add code here that assigns an appropriate value to score
        
        return "N" if score < 0 else "P" 

    def batch_classify(self, docs): 
        return [self.classify(doc.words() if hasattr(doc, 'words') else doc) for doc in docs] 

    def labels(self): 
        return ("P", "N")

#Example usage:

classifier = SimpleClassifier(top_pos, top_neg)
classifier.classify("I read the book".split())

'P'

### Exercise

- Copy the above code cell and move it to below this one. Then complete the `classify` method in the above code as specified below.
- Test your classifier on several very simple hand-crafted examples to verify that you have implemented `classify` correctly.

The classifier is initialised with a list of positive words, and a list of negative words. The words of a document are passed to the `classify` method (which is partially completed in the above code fragment). The `classify` method should be defined so that each occurrence of a negative word decrements `score`, and each occurrence of a positive word increments `score`. 
- For `score` less than 0, an "`N`" for negative should be returned.
- For `score` greater than 0,  "`P`" for positive should returned.
- For `score` of 0, the classification decision should be made randomly (see https://docs.python.org/3/library/random.html).


In [80]:
from nltk.classify.api import ClassifierI
import random

class SimpleClassifier(ClassifierI):

    def __init__(self, pos, neg): 
        self._pos = pos 
        self._neg = neg 

    def classify(self, words): 
        score = 0
        for word in words:
            if word in self._pos:
                score += 1
            elif word in self._neg:
                score -= 1   
            print(score)
        if score > 0:
            return "P"
        if score < 0:
            return "N"        
        return random.choice(["N", "P"])
        
    def batch_classify(self, docs): 
        return [self.classify(doc.words() if hasattr(doc, 'words') else doc) for doc in docs] 

    def labels(self): 
        return ("P", "N")    

#Example usage:

classifier = SimpleClassifier(top_pos, top_neg)
classifier.classify("good I bad".split())

1
2
1


'P'

In [74]:
# %load solutions/simple_classify

## Evaluating word list based classifier
Below is code that uses an evaluation function in order to determine how well your classifier performs. The function returns the <b>accuracy</b> of a classifier. The accuracy metric is defined as the proportion of documents that were correctly classified.

In [22]:
from sussex_nltk.stats import evaluate_wordlist_classifier

#Create a new classifier with your words lists
book_classifier = SimpleClassifier(top_pos, top_neg)

#Evaluate classifier
#The function requires three arguments:
# 1. Word list based classifer
# 2. A list (or generator) of positive AmazonReview objects
# 3. A list (or generator) of negative AmazonReview objects
score = evaluate_wordlist_classifier(book_classifier, pos_test, neg_test)  
print(score)

0.6143939393939394


### Exercise  

Use the blank cell below to perform the following two experiments:
- Evaluate the performance of a classifier using hand-crafted lists.
- Evaluate the performance of a classifier using lists derived from the training data.

In [29]:
# %load solutions/classifier_test
from sussex_nltk.stats import evaluate_wordlist_classifier

experiments = [("Hand-Crafted lists",my_positive_word_list,my_negative_word_list),
               ("Top-k lists",top_pos,top_neg),
               ("Above-k lists",above_pos,above_neg)]


for description,pos_list,neg_list in experiments:
    #Create a new classifier with your words lists
    classifier = SimpleClassifier(pos_list, neg_list)
    score = evaluate_wordlist_classifier(classifier, pos_test, neg_test)
    print("The accuracy of the {0} classifer is {1:.2f}".format(description,score))


The accuracy of the Hand-Crafted lists classifer is 0.57
The accuracy of the Top-k lists classifer is 0.50
The accuracy of the Above-k lists classifer is 0.50


## Naïve Bayes classifiers
We now turn to a different model of document classification known as Naïve Bayes.

Before we apply them to Amazon reviews, we need to implement them!

We will introduce them through a very simple example dataset involving documents that are either about the weather or football. The classifier will be trained to distinguish these two topics from one another.

There are, therefore, two classes `weather` and `football`. The classifier's job is to determine whether a document that it is given is in the class `weather` or in the class `football`.

We give the classifier examples of documents in the `weather` class, and examples of documents in the `football` class. For now, to keep things simple, our so-called documents will be very short phrases.

Run the following cell to set up some training data.

In [30]:
# Some test data to get us started. It is deliberately VERY simple. 

weather_sents_train = [
    "today it is raining",
    "looking cloudy today",
    "it is nice weather",
]

football_sents_train = [
    "city looking good",
    "advantage united",
] 

### Bag of words
One of the simplying assumptions that is made with Naïve Bayes classification is that each document is taken to be a so-called bag of words. What this means is that the word order is ignored, and we are not taking into account the number of times a word appears in a document. Note that there are variants of Naïve Bayes where the number of times a words appears in a document is taken into account, but we will not consider that case here.

Given the bag-of-words assumption, we will represent each training document as a pair consisting of a dictionary that maps each word that appears in the document to `True`, and a string denoting the class of the document. 

### Exercise
In the cell below, write code that achieves this. Create three dictionaries:
- `weather_data_train`: a dictionary containing the data for documents in the class `weather`;
- `football_data_train`: a dictionary containing the data for documents in the class `football`;
- `train_data`: which is simply `weather_data_train + football_data_train`

Hint: this can be done with nested list comprehensions.

In [62]:
def make_dict(data):
    data_train = []
    for doc in data:
        word_true = {}
        for word in doc.split():
            word_true[word] = True
        if doc in weather_sents_train:
            data_train.append((word_true, "weather"))
        else:
            data_train.append((word_true, "football"))
    return data_train
    
train_data = make_dict(weather_sents_train) + make_dict(football_sents_train)
train_data


[({'is': True, 'it': True, 'raining': True, 'today': True}, 'weather'),
 ({'cloudy': True, 'looking': True, 'today': True}, 'weather'),
 ({'is': True, 'it': True, 'nice': True, 'weather': True}, 'weather'),
 ({'city': True, 'good': True, 'looking': True}, 'football'),
 ({'advantage': True, 'united': True}, 'football')]

In [61]:
# %load solutions/format_data

### How a Naïve Bayes classifier works

We will now look at how a NB work, using our `weather` versus `football` classification task.

In order to classify a document, $d$, we need to determine which of these probabilities is greatest:

$$P(\,\mbox{weather}\,|\,d) \qquad\qquad \mbox{versus} \qquad\qquad P(\,\mbox{football}\,|\,d)$$

$d$ could, for example, be the string "today is looking cloudy", which would give us:

$$P(\,\mbox{weather}\,|\,\mbox{"today is looking cloudy"}) \qquad\qquad \mbox{versus} \qquad\qquad P(\,\mbox{football}\,|\,\mbox{"today is looking cloudy"})$$

The idea is that if the term on the left is higher then the document is in category `weather`, and if the term on the right is higher then the document is in category `football`.

$P(X|Y)$ means the probability of $X$ given $Y$. So, $P(\,\mbox{weather}\,|\,d)$ means the probability, given a document $d$, of $d$ being of class `weather`.

We are going to use something called Bayes' rule which states that:

$$P(X|Y) = \frac{P(Y|X)\cdot P(X)}{P(Y)}$$

Applying Bayes' rule to our problem leads to the following comparision:

$$\frac{P(\,d\,|\,\mbox{weather}\,)\cdot P(\,\mbox{weather}\,)}{p(d)} \qquad\qquad \mbox{versus} \qquad\qquad \frac{P(\,d\,|\,\mbox{football}\,)\cdot P(\,\mbox{football}\,)}{p(d)}$$

Since both sides are being divided by the same thing, we only need to make the following comparision:

$$P(\,d\,|\,\mbox{weather}\,)\cdot P(\,\mbox{weather}\,) \qquad\qquad \mbox{versus} \qquad\qquad P(\,d\,|\,\mbox{football}\,)\cdot P(\,\mbox{football}\,)$$

Let's just look at what each of these probabilities mean?

1. $P(\,d\,|\,\mbox{weather}\,)$: this is the probability of a document in the `weather` category being the document $d$

2. $P(\,d\,|\,\mbox{football}\,)$: this is the probability of a document in the `football` category being the document $d$

3. $P(\,\mbox{weather}\,)$: this is the probability of a randomly selected document being of category `weather`.

4. $P(\,\mbox{football}\,)$: this is the probability of a randomly selected document being of category `football`.

How are we going to obtain these probabilities? 


### Class priors

We have established that we need to know $P(\,\mbox{weather}\,)$ and $P(\,\mbox{football}\,)$. These are called the class priors. Let's see how we can obtain (estimated) values for these probabilities from our training data.

The classifier has seen three documents of class `weather` and two documents of class `football`.

From this it learns that `weather` documents are slightly more common that `football` documents, and it therefore has a slight bias towards saying a document is a `weather` document.

To be more precise, the classifier has learned that:
- The probability that a document is in class `weather` is $3/5$.  
We say $P(\mbox{weather})=3/5=0.6$.

- The probability that a document is in class `football` is $2/5$.  
We say $P(\mbox{football})=2/5=0.4$.

In general, if the training data contained $n_1$ documents of class `weather` and $n_2$ documents of class `football`, then 

$$P(\mbox{weather})=\frac{n_1}{n_1+n_2} \qquad \mbox{and} \qquad P(\mbox{football})=\frac{n_2}{n_1+n_2}$$

### Exercise
In blank the cell below, implement a function `class_priors(training_data)` that takes a dictionary of training data  and returns a dictionary that maps the name of each class to the class prior for that class.

Once you have done this, test it out on the training data above.

In [67]:
def class_priors(training_data):
    docs_per_class = collections.defaultdict(int)
    priors = collections.defaultdict(float)
    for doc, c in training_data:
        docs_per_class[c] += 1
    for c in docs_per_class.keys():
        priors[c] = docs_per_class[c]/len(training_data)
    return priors
class_priors(train_data)

defaultdict(float, {'football': 0.4, 'weather': 0.6})

In [68]:
# %load solutions/class_priors

### Conditional probabilities
We now turn to the problem of how to calculate (estimates of) the probabilities such as $P(\,d\,|\,\mbox{weather}\,)$ and $P(\,d\,|\,\mbox{football}\,)$, for some document $d$. The problem we have is that $d$ is a document, potentially a long document, that we won't have seen in the training data.

To address this, the Naïve Bayes model of document classification makes a major simplifying assumption.  In particular, it is assumed that the probabiity that different words occur in a document are independent of one another.

For example, if $d=\mbox{"today is looking cloudy"}$ then this assumption tells us that:

\begin{eqnarray*}
P(\,\mbox{"today is looking cloudy"}\,|\,\mbox{weather}\,) &=& P(\{\mbox{"today"},\mbox{"is"},\mbox{"looking"},\mbox{"cloudy"}\}\,|\,\mbox{weather}\,)\\
&=& P(\,\mbox{"today"}\,|\,\mbox{weather}\,)\times P(\mbox{"is"}\,|\,\mbox{weather}\,)\times P(\mbox{"looking"}\,|\,\mbox{weather}\,)\times P(\mbox{"cloudy"}\,|\,\mbox{weather}\,)
\end{eqnarray*}

For the general case, with class $c$ and document $d=\{w_1,\ldots,w_n\}$, we have:
\begin{eqnarray*}
P(\,d\,|\,c\,) &=& P(\,\{w_1,\ldots,w_n\}\,|\,c\,)\\
&=& \prod_{i=1}^n P(\,w_i\,|\,c\,)
\end{eqnarray*}

The point is that it is plausible that given a reasonable amount of training data, we can make reasonable estimates of the probabilities $P(\,\mbox{"today"}\,|\,\mbox{weather}\,)$, $P(\mbox{"is"}\,|\,\mbox{weather}\,)$, $P(\mbox{"looking"}\,|\,\mbox{weather}\,)$, and  $P(\mbox{"cloudy"}\,|\,\mbox{weather}\,)$. 

We not look at how that is done.

### Estimating conditional probabilities
So we have established that we need estimates of probabilities such as: $P(\,\mbox{"cloudy"}\,|\,\mbox{weather}\,)$, $P(\mbox{"is"}\,|\,\mbox{weather}\,)$, $P(\mbox{"today"}\,|\,\mbox{weather}\,)$, and $P(\mbox{"looking"}\,|\,\mbox{weather}\,)$.

How can these probabilities be estimated from the training data?

Look at the training data we set up above. There are 3 documents of class `weather` and within these documents there are a total of 11 tokens (8 different types).

From the data we can estimate the following probabilities:
- the probability of seeing "today" in a `weather` document is $\frac{2}{11}$,  
i.e. $P(\mbox{"today"}\,|\,\mbox{weather})=\frac{2}{11}$;
- the probability of seeing "it" in a `weather` document is $\frac{2}{11}$,  
i.e. $P(\mbox{"it"}\,|\,\mbox{weather})=\frac{2}{11}$;
- the probability of seeing "is" in a `weather` document is $\frac{2}{11}$,  
i.e. $P(\mbox{"is"}\,|\,\mbox{weather})=\frac{2}{11}$;
- the probability of seeing "raining" in a `weather` document is $\frac{1}{11}$,  
i.e. $P(\mbox{"raining"}\,|\,\mbox{weather})=\frac{1}{11}$;
- the probability of seeing "looking" in a `weather` document is $\frac{1}{11}$,  
i.e. $P(\mbox{"looking"}\,|\,\mbox{weather})=\frac{1}{11}$;
- the probability of seeing "cloudy" in a `weather` document is $\frac{1}{11}$,  
i.e. $P(\mbox{"cloudy"}\,|\,\mbox{weather})=\frac{1}{11}$;
- the probability of seeing "nice" in a `weather` document is $\frac{1}{11}$,  
i.e. $P(\mbox{"nice"}\,|\,\mbox{weather})=\frac{1}{11}$; and
- the probability of seeing "weather" in a `weather` document is $\frac{1}{11}$,  
i.e. $P(\mbox{"weather"}\,|\,\mbox{weather})=\frac{1}{11}$
- the probability of seeing any other word in a `weather` document is 0.
Notice that all of these conditional probabilities sum to $1$.

We can do the same thing for the `football` documents:
- the probability of seeing "city" in a `football` document is $\frac{1}{5}$,  
i.e. $P(\mbox{"city"}\,|\,\mbox{weather})=\frac{1}{5}$;
- the probability of seeing "looking" in a `football` document is $\frac{1}{5}$,  
i.e. $P(\mbox{"looking"}\,|\,\mbox{weather})=\frac{1}{5}$;
- the probability of seeing "good" in a `football` document is $\frac{1}{5}$,  
i.e. $P(\mbox{"good"}\,|\,\mbox{weather})=\frac{1}{5}$;
- the probability of seeing "advantage" in a `football` document is $\frac{1}{5}$,  
i.e. $P(\mbox{"advantage"}\,|\,\mbox{weather})=\frac{1}{5}$;
- the probability of seeing "united" in a `football` document is $\frac{1}{5}$,  
i.e. $P(\mbox{"united"}\,|\,\mbox{weather})=\frac{1}{5}$;
- the probability of seeing any other word in a `football` document is 0.


### Exercise
We now look at how to implement the calculation of conditional probabilties.

In the empty cell below, define a function `cond_probs(training_data)` that takes training data and returns a dictionary that maps the name of a class, $c$, onto a dictionary that maps each word, $w$ to the conditional probability for that word given that class, i.e. $\log(P(w|c))$.

Hint: the following line will create a dictionary of the required form:  
`c_probs = collections.defaultdict(lambda: defaultdict(float))`

In [70]:
def cond_probs(training_data):
    c_probs = collections.defaultdict(lambda: defaultdict(float))
    words_per_doc = collections.defaultdict(int)
    for doc, c in training_data:
        words_per_doc[c] += len(doc)
        for word in doc:
            c_probs[c][word] += 1
    for c in words_per_doc:
        for word in c_probs[c]:
             c_probs[c][word] /= words_per_doc[c]            
    return c_probs
    
c_ps = cond_probs(train_data)

c_ps["weather"]["cloudy"]

0.09090909090909091

In [10]:
# %load solutions/cond_probs_partial

In [69]:
# %load solutions/cond_probs

### Computing the conditional probability of a document

We have created implementations of the following functions:
- `class_priors(training_data)` that computes estimates of the class priors from training data;
- `cond_probs(training_data)` that computes estimates of the conditional probability of a word given a class from training data

Let us suppose that we have applied these functions to our training data as follows.

```
c_priors = class_priors(train_data)
c_probs = cond_probs(train_data)
```

`c_priors` and `c_probs` define the classifier.

### Exercise
In the cell below, complete the function `classify(doc,c_priors,c_probs)`. It should return the class that the classifier assigns to the document, `doc`. 

In the event of a tie, the function should randomly chose one of the classes (see `random.choice`). 

- Write your function in a way that allows for the possibilty of any number of classes.
- Assume that the document, `doc`, is represented as a dictionary that maps words (in the document) to `True`.

In [124]:
def classify(doc,priors,c_probs):
    class_scores = collections.defaultdict(lambda:1)
    for c in priors.keys():
        class_scores[c] *= priors[c]
        for word in doc:
            class_scores[c] *= c_probs[c][word]
    print(class_scores)
    best = max(class_scores.values())
    return random.choice([c for c in class_scores.keys() if class_scores[c] == best])

c_priors = class_priors(train_data)
c_probs = cond_probs(train_data)
sent = "looking cloudy today"
doc = dict([(word, True) for word in sent.split()])
classify(doc,c_priors,c_probs)

defaultdict(<function classify.<locals>.<lambda> at 0x000001636E5B8400>, {'weather': 0.09231527903345112, 'football': 0.0122124974557297})


'weather'

In [89]:
# %load solutions/my_NB_classify

### A problem
There is a problem with the classifier that we have written. 

### Exercise
Run the classifier on several examples to see if you can discover what the problem is. 
- You might need to run the same document through the classifier several times to see if you get different class for different runs
 - If you've implemented `classify` correctly then this happens when you have a tie.
- Try the sentence "city looking cloudy today"

What is going wrong?

### Add one smoothing
It will often be the case that a word appears in documents of one class, but not in any documents of another class.
- We saw that with the word "city" in the document "city looking cloudy today".
- While "city" appears in documents of class `football`, it does not appear in any documents of class `weather`. 
- Thus, the conditional probabiity $P(\,\mbox{city}\,|\,\mbox{weather}\,)$ is equal to zero.
- We are **multiplying** probabilities, so the document ends up with a score of zero even though all of the other words in the document suggest that it is of class `weather`.

To get around this we need to do something called **smoothing** in order to avoid zero probabilities. 

In particular, we will implement a version of smoothing called **add-one smoothing**. This involves adding a count of one to all of the known vocabulary.

### Known vocabulary
The words that appear in documents in the training data are collectively described as the known vocabulary. These are the words that the classifier can learn something about. If the classifier is asked to classify a document that contains any words that are not in the known vocabulary then the classifier will simply ignore them.

### Exercise
In the cell below, write a function `known_vocabulary(training_data)` that takes some training and returns a set containing all of words that appear in documents in the training data.

In [92]:
def known_vocabulary(training_data):
    known_voc = set()
    for doc, c in training_data:
        for word in doc.keys():
            known_voc.add(word)
    return known_voc 

known_vocabulary(train_data)

{'advantage',
 'city',
 'cloudy',
 'good',
 'is',
 'it',
 'looking',
 'nice',
 'raining',
 'today',
 'united',
 'weather'}

In [93]:
# %load solutions/vocabulary


### Implementing add-one smoothing
As the name suggests, add-one smoothing involves adding counts.

In particular, for each word, $w$, in the known vocabulary and each class, $c$, we add one extra count to our record of how many times $w$ appears in documents of class $c$. We are, in effect, hallucinating counts. The reason for doing this is that it means that we avoid zero probabilties.

### Exercise
In the blank cell below copy in your code for the `cond_probs` function that you wrote earlier. Then adapt this code so that it implements the add-one smoothing scheme.
- You will find it useful to use your `known_vocabulary` function.
- If there are $k$ words in your known vocabulary, then you will add $k$ counts for each class. 
- Therefore, when calculating conditional probabilities, you need to add $k$ to the denominator to account for these extra counts.

Test out your code.

In [115]:
def cond_probs(training_data):
    c_probs = collections.defaultdict(lambda: defaultdict(float))
    words_per_doc = collections.defaultdict(int)
    voc = known_vocabulary(training_data)
    for doc, c in training_data:
        words_per_doc[c] += len(doc)
        for word in doc:
            c_probs[c][word] += 1
            for word in voc:
                c_probs[c][word] += 1
    for c in words_per_doc:
        words_per_doc[c] += len(voc)
    for c in words_per_doc.keys():
        for word in c_probs[c].keys():
             c_probs[c][word] /= words_per_doc[c]        
    return c_probs

c_ps = cond_probs(train_data)

c_ps["weather"]["city"]

0.4782608695652174

In [132]:
# %load solutions/smoothed_cond_probs
def cond_probs(training_data):
    # c_probs will hold our conditional probabilities
    c_probs = collections.defaultdict(lambda: collections.defaultdict(float)) 
    # docs_with_word is a mapping from a class to a mapping from a word to number of documents of that category the word appeared in 
    docs_with_word = collections.defaultdict(lambda: collections.defaultdict(int)) 
    # tot_words is a mapping from a class to the total number of words documents of that class
    tot_words = collections.defaultdict(int)  
    
    # first get the counts of words in documents of a class and total word count per class
    for doc,c in training_data:
        for word in doc:
            docs_with_word[c][word] += 1
            tot_words[c] += 1
    # next, add the add-one smoothing counts
    known_vocab = known_vocabulary(training_data)
    for c in docs_with_word.keys():
        for word in known_vocab:
            docs_with_word[c][word] += 1
    # update tot_words to account for the additional (hallucinated) counts
        tot_words[c] += len(known_vocab)
    # now compute the conditional probabilities
    for c in docs_with_word.keys(): 
        for word in docs_with_word[c].keys():
            c_probs[c][word] = docs_with_word[c][word] / tot_words[c]
    return c_probs
        
cond_probs(train_data)

c_ps["weather"]["city"]


0.4782608695652174

### Smoothing class priors
It is possible (though perhaps unlikely) that the training data does not contain any data for one class. In order to take care of this, we also smooth the class priors.

### Exercise
In the cell below, revise the `class_priors` function so that it adds one to the count of each class.

Check that the sum of the priors for each of the classes is $1$.

In [133]:
def class_priors(training_data):
    priors = {}
    counts = collections.defaultdict(int)
    for doc, c in training_data:
        counts[c] += 1
    for c in counts.keys():
        counts[c] += 1
    for c in counts.keys():
        priors[c] = counts[c]/(len(training_data) + len(counts.items()))
    return priors

class_priors(train_data)

{'football': 0.42857142857142855, 'weather': 0.5714285714285714}

In [134]:
# %load solutions/smoothed_class_priors
def class_priors(data):
    doc_counts = collections.defaultdict(int)
    priors = collections.defaultdict(float)
    # first we get the document count for each class
    for doc,c in data:
        doc_counts[c] += 1
    # now we add counts to achieve add-one smoothing
    for c in doc_counts:
        doc_counts[c] += 1
    # now we compute the probabilities 
    # we must add len(doc_counts) to the denominator because of the add-one smoothing
    for c in doc_counts.keys():
        priors[c] = doc_counts[c]/(len(data)+len(doc_counts)) 
    return priors

priors = class_priors(train_data)
print("The prior for class 'football' is {0:.3f}.\nThe prior for class 'weather' is {1:.3f}.\nThe priors sum to {2:.3f}".
      format(priors["football"],priors["weather"],priors["football"] + priors["weather"]))


The prior for class 'football' is 0.429.
The prior for class 'weather' is 0.571.
The priors sum to 1.000


### Ignoring OOV words
We now look at how to update the `classify` function that we wrote earlier so that it ignores out of vocabulary words that appear in a document being classified.

### Exercise
In the blank cell below, copy the `classify` method you wrote earlier and update it so that words not in the known vocabulary are ignored.
- You will want to add an additional argument to the `classify` function that is a set containing the known vocabulary.

In [155]:
def classify(doc,priors,c_probs, voc):
    class_scores = collections.defaultdict(lambda:1)
    for c in priors.keys():
        class_scores[c] *= priors[c]
        for word in doc:
            if word in voc:
                class_scores[c] *= c_probs[c][word]
    best = max(class_scores.values())
    return random.choice([c for c in class_scores.keys() if class_scores[c] == best])    

c_priors = class_priors(train_data)
c_probs = cond_probs(train_data)
voc = known_vocabulary(train_data)
sent = "looking good today" 
doc = dict([(word, True) for word in sent.split()])
classify(doc,c_priors,c_probs, voc)

'football'

In [156]:
# %load solutions/new_classify
def classify(doc,priors,c_probs,known_vocab):
    class_scores = collections.defaultdict(lambda:1)
    for c in priors.keys():
        class_scores[c] *= priors[c]
        for word in doc:
            if word in known_vocab:
                class_scores[c] *= c_probs[c][word]
    best_score = max(class_scores.values())
    return random.choice([c for c in class_scores.keys() if class_scores[c]== best_score])

c_priors = class_priors(train_data)
c_probs = cond_probs(train_data)
known_vocab = known_vocabulary(train_data)
sent = "looking really cloudy today"
doc = dict([(word, True) for word in sent.split()])
classify(doc,c_priors,c_probs,known_vocab)

'weather'

### Underflow

We need to address one final problem concerning the multiplication of probabilities.

Recall this equation from earlier:

\begin{eqnarray*}
P(\,d\,|\,c\,) &=& P(\,\{w_1,\ldots,w_n\}\,|\,c\,)\\
&=& \prod_{i=1}^n P(\,w_i\,|\,c\,)
\end{eqnarray*}

This tells us that in order to compute $P(\,d\,|\,c\,)$ for some document $d$ and class $c$, we must multiply $n$ conditional probabilities, one for each word in the document.

While in our toy example, this is not an issue. However, in a more realistic settings, where we had thousands of documents, each of which contained multiple paragraphs, we would find ourselves multiplying large numbers of very small probabilities. This would lead to **underflow**.

To avoid ths problem, we will add the log of probabilties. 

To understand why this is a reasonable thing to do let us recall a comparison from earlier:

$$P(\,d\,|\,\mbox{weather}\,)\cdot P(\,\mbox{weather}\,) \qquad\qquad \mbox{versus} \qquad\qquad P(\,d\,|\,\mbox{football}\,)\cdot P(\,\mbox{football}\,)$$

Our goal is to determine which of the values (on the left and right) is larger (or determine that they are equal). 

It should be clear that we will get exactly the same answer to this question by making the following comparsion.

$$\log(P(\,d\,|\,\mbox{weather}\,)) + \log(P(\,\mbox{weather}\,)) \qquad\qquad \mbox{versus} \qquad\qquad \log(P(\,d\,|\,\mbox{football}\,)) + \log(P(\,\mbox{football}\,))$$

Thus, rather than calculating conditional probabilities as described above, we will calculate log conditional probabilities like this:

\begin{eqnarray*}
\log(P(\,\mbox{"today is looking cloudy"}\,|\,\mbox{weather}\,)) &=& \log(P(\{\mbox{"today"},\mbox{"is"},\mbox{"looking"},\mbox{"cloudy"}\}\,|\,\mbox{weather}\,))\\
&=& \log(P(\,\mbox{"today"}\,|\,\mbox{weather}\,))\ +\\
&&\log(P(\mbox{"is"}\,|\,\mbox{weather}\,))\ +\\
&&\log(P(\mbox{"looking"}\,|\,\mbox{weather}\,))\ + \\
&&\log(P(\mbox{"cloudy"}\,|\,\mbox{weather}\,))
\end{eqnarray*}

For the general case, with class $c$ and document $d=\{w_1,\ldots,w_n\}$, we have:

\begin{eqnarray*}
\log(P(\,d\,|\,c\,)) &=& \log(P(\,\{w_1,\ldots,w_n\}\,|\,c\,))\\
&=& \sum_{i=1}^n \log(P(\,w_i\,|\,c\,))
\end{eqnarray*}


### Exercise
In the blank cell below, make a copy of the cell containing the definition of `classify`.

Adapt the code so that it adds logs of probabilties rather than multiplies probabilities.

In [66]:
def classify(doc, priors, c_probs, voc):
    class_scores = collections.defaultdict(lambda:1)
    for c in priors.keys():
        class_scores[c] += math.log(priors[c])
        for word in doc:
            if word in voc:
                class_scores[c] += math.log(c_probs[c][word])
    best = max(class_scores.values())
    return random.choice([c for c in class_scores.keys() if class_scores[c] == best])    

c_priors = class_priors(train_data)
c_probs = cond_probs(train_data)
voc = known_vocabulary(train_data)
sent = "looking good today" 
doc = dict([(word, True) for word in sent.split()])
classify(doc,c_priors,c_probs, voc)

'weather'

In [157]:
# %load solutions/classify_final
def classify(doc,priors,c_probs,known_vocab):
   class_scores = collections.defaultdict(lambda:0)
   for c in priors.keys():
       class_scores[c] += math.log(priors[c])
       for word in doc:
           if word in known_vocab:
               class_scores[c] += math.log(c_probs[c][word])
   best_score = max(class_scores.values())
   return random.choice([c for c in class_scores.keys() if class_scores[c]== best_score])

c_priors = class_priors(train_data)
c_probs = cond_probs(train_data)
known_vocab = known_vocabulary(train_data)
sent = "city looking cloudy today"
doc = dict([(word, True) for word in sent.split()])
classify(doc,c_priors,c_probs,known_vocab)


'weather'

### Evaluating a Naïve Bayes classifier on test data
We are now ready to run our Naïve Bayes classifier on a set of test data. When we do this we want to return the accuracy of the classifier on that data, where accuracy is calculated as follows:

$$\frac{\mbox{number of test documents that the classifier classifiers correctly}}
{\mbox{total number of test documents}}$$

In order to compute this accuracy score, we need to give the classifier **labelled** test data.
- This will be in the same format as the training data.

### Exercise
In the cell below, we set up 5 test documents in the class `weather` and 5 documents in the class `football`.

Run this cell.

In [178]:
weather_sents_train = [
    "today it is raining",
    "looking cloudy today",
    "it is nice weather",
]

football_sents_train = [
    "city looking good",
    "advantage united",
]

weather_data_train = [(dict([(word, True) for word in sent.split()]), "weather") for sent in weather_sents_train] 
football_data_train = [(dict([(word, True) for word in sent.split()]), "football") for sent in football_sents_train]
train_data = weather_data_train + football_data_train

weather_sents_test = [
    "the weather today is nice",
    "it is raining cats and dogs",
    "the weather here is wet",
    "goal it was hot today",
    "rain due tomorrow"
]

football_sents_test = [
    "what a great goal that was",
    "poor defending by the city center back",
    "wow he missed a sitter",
    "united are a shambles",
    "shots raining down on the keeper",
]

weather_data_test = [(dict([(word, True) for word in sent.split()]), "weather") for sent in weather_sents_test] 
football_data_test = [(dict([(word, True) for word in sent.split()]), "football") for sent in football_sents_test]
test_data = weather_data_test + football_data_test



### Exercise
In the cell below Implement a `NB_evaluate` function that returns the accuracy of a classifier on a set of labelled test data.
`NB_evaluate` should make use of the `classify` function, and take the following arguments:
- the test data
- the class priors
- the conditional probabilities
- the known vocabulary (though this is redundant since it could be computed from the conditional probabilities)

`NB_evaluate` should return the accuracy of the classifier on the test data.

Try out your `NB_evaluate` function on the test data in the cell above.

In [179]:
def NB_evaluate(test_data, priors, c_probs, voc):
    correct = 0
    for doc, c in test_data:
        pred_class = classify(doc, priors, c_probs, voc)
        print(doc)
        print(pred_class)
        if pred_class == c:
            correct += 1            
    return correct/len(test_data)

NB_evaluate(test_data, priors, c_probs, known_vocab)

{'the': True, 'weather': True, 'today': True, 'is': True, 'nice': True}
weather
{'it': True, 'is': True, 'raining': True, 'cats': True, 'and': True, 'dogs': True}
weather
{'the': True, 'weather': True, 'here': True, 'is': True, 'wet': True}
weather
{'goal': True, 'it': True, 'was': True, 'hot': True, 'today': True}
weather
{'rain': True, 'due': True, 'tomorrow': True}
weather
{'city': True}
football
{'poor': True, 'defending': True, 'by': True, 'the': True, 'city': True, 'center': True, 'back': True}
football
{'wow': True, 'he': True, 'missed': True, 'a': True, 'sitter': True}
weather
{'united': True, 'are': True, 'a': True, 'shambles': True}
football
{'shots': True, 'raining': True, 'down': True, 'on': True, 'the': True, 'keeper': True}
weather


0.8

In [164]:
# %load solutions/NB_evaluate
def NB_evaluate(test_data,priors,c_probs,known_vocab):
    num_correct = 0
    for doc,c in test_data:
        predicted_class = classify(doc,priors,c_probs,known_vocab)
        if predicted_class == c:
            num_correct += 1
    return num_correct/len(test_data)

NB_evaluate(test_data,priors,c_probs,known_vocab)


0.7272727272727273

### Exercise
Try out the classifier on different training and test examples.

Before running each test try to anticipate what you think the outcome will be.
- What happens when none of the words in the test document are in the known vocabulary?
- Can you set up data so that there is a tie between the classes, and the classifier randomly chooses a class. 